In [1]:
#!pip3 install syntok
#!pip3 install tqdm
#!pip3 install pandas
#!pip3 install ipywidgets

In [2]:
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
from zipfile import ZipFile
import syntok.segmenter as segmenter
import re

In [3]:
path = Path('../../Lab1/data/hmd/plaintext')
#path_metadata = Path('../../../../LivingwithMachines/Lab1/data/hmd/metadata')
# git 
save_to = Path('hmd_data_extension_words')
save_to.mkdir(exist_ok=True)

In [9]:
#pattern = re.compile(r'(?:\bmachine[s]{,1}\b|\bengine[s]{,1}\b)',re.I)
#pattern.findall('machine Machines Mdachine dmachines')
#pattern = re.compile(r'(?:\bwom[ea]n\b|\bboy[s]{,1}\b|\bart[ie][zs]an\b|\bslave[s]{,1}\b)')
#pattern = re.compile(r'(?:\bgirl[s]{,1}\b|\bart[ie][zs]an[s]{,1}\b)')
pattern = re.compile(r'(?:\baccident[s]{,1}\b)',re.I)

In [10]:
pattern.findall('accident accidents cbaccident Accident')

['accident', 'accidents', 'Accident']

In [11]:
zip_files = list(path.glob('*.zip'))
len(zip_files)

14

In [12]:
def harvest_sentences(text,path, regex_pattern):
    if regex_pattern.findall(text):
        sentences = [' '.join([t.value for t in s]) 
                            for p in segmenter.process(text) for s in p
                                ]
        sentences = ["<START>"] + sentences + ["<END>"]
        
        target_sents = [(path,
                        sentences[i-1],
                        sentences[i],
                        sentences[i+1],
                        '<SEP>'.join(
                                set(regex_pattern.findall(s)))
                                     )
                                    for i,s in enumerate(sentences) 
                                             if regex_pattern.findall(s)
                                ]
        return target_sents
    return None

In [13]:
count = 0

for zf in zip_files:
    with ZipFile(zf, 'r') as hmd_zip:
        articles = hmd_zip.namelist()
        rows = []
        for art in tqdm(articles):
            if not art.endswith('.txt'): continue
            text = hmd_zip.read(art).decode("utf-8") 
            target_sentences = harvest_sentences(text,art,regex_pattern=pattern)
            if target_sentences:
                rows.extend(target_sentences)
                count+=len(target_sentences)
                
        df = pd.DataFrame(rows,columns=['article_path','previous_sentence','target_sentence','next_sentence','hits'])
        df.to_csv(save_to / (zf.stem + '.csv'))
                    
print(count)

  0%|          | 0/246152 [00:00<?, ?it/s]

  0%|          | 0/1025 [00:00<?, ?it/s]

  0%|          | 0/25434 [00:00<?, ?it/s]

  0%|          | 0/30532 [00:00<?, ?it/s]

  0%|          | 0/203386 [00:00<?, ?it/s]

...

  0%|          | 0/96414 [00:00<?, ?it/s]

  0%|          | 0/38578 [00:00<?, ?it/s]

  0%|          | 0/1625117 [00:00<?, ?it/s]

  0%|          | 0/11844 [00:00<?, ?it/s]

182757


In [ ]:
!open {save_to}